# 0. Imports

In [19]:
%load_ext autoreload
%autoreload 2

# imports
import pandas as pd
import numpy as np
import datetime as dt
import time
import os
import pickle

from colorama import Fore, Style
from sklearn.pipeline import Pipeline
from joblib import dump, load

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Importing those functions from the API package so they can be used
# by the loaded pipeline in the API package later
from decipherer.ml_logic.encoders import ffill_nan, add_datetime_features

In [20]:
# environment variables
LOCAL_REGISTRY_PATH = "../training_outputs"

# prerequisite: having a "training_outputs" folder at the root of the project
! tree ../training_outputs

../training_outputs
├── metrics
│   ├── 20221123-163916.pickle
│   ├── 20221123-170451.pickle
│   ├── 20221124-115822.pickle
│   ├── 20221125-124411.pickle
│   ├── 20221125-141247.pickle
│   ├── 20221125-141615.pickle
│   └── 20221125-143102.pickle
├── models
│   ├── 20221123-163916.joblib
│   ├── 20221123-170451.joblib
│   ├── 20221124-115822.joblib
│   ├── 20221125-124411.joblib
│   ├── 20221125-141247.joblib
│   ├── 20221125-141615.joblib
│   └── 20221125-143102.joblib
└── params
    ├── 20221123-163916.pickle
    ├── 20221123-170451.pickle
    ├── 20221124-115822.pickle
    ├── 20221125-124411.pickle
    ├── 20221125-141247.pickle
    ├── 20221125-141615.pickle
    └── 20221125-143102.pickle

3 directories, 21 files


# 1. Build pipeline

In [21]:
# Get rid of nan values using ffil
# def ffill_nan(X):
#     return X.fillna(method='ffill', axis = 0)

In [22]:
# # Create new datetime features
# def add_datetime_features(X):
    
#     # Copy X to avoid pandas warning
#     X_rep = X.copy()
    
#     # Handle datetime format
#     datetime = pd.to_datetime(X_rep['date'] + ' ' + X_rep['time'], format="%d/%m/%Y %H:%M:%S")
    
#     # Create new features using month, weekday, hour and minute
#     X_rep['month'] = datetime.dt.month
#     X_rep['weekday'] = datetime.dt.weekday
#     X_rep['hour'] = datetime.dt.hour
#     X_rep['minute'] = datetime.dt.minute
    
#     # Consider periodic effects
#     X_rep['month_sin'] = np.sin(2*np.pi*X_rep['month']/12)
#     X_rep['month_cos'] = np.cos(2*np.pi*X_rep['month']/12)
    
#     # Get rid of Datetime
#     return X_rep.drop(columns=['date', 'time'])

In [23]:
# Create a pipeline to preprocess the data

n_estimators = 10

features = ['global_active_power', 'global_reactive_power', 'voltage', 'global_intensity', 'global_consumption']
datetimes = ['date', 'time']

preparator = ColumnTransformer([ 
    ('imputer', FunctionTransformer(ffill_nan), features),
    ('datetime_features_adder', FunctionTransformer(add_datetime_features), datetimes)
])

pipeline = Pipeline([
    ('preparator', preparator),
    ('std_scaler', StandardScaler()),
    ('estimator', RandomForestRegressor(n_estimators=n_estimators))
])

pipeline

Pipeline(steps=[('preparator',
                 ColumnTransformer(transformers=[('imputer',
                                                  FunctionTransformer(func=<function ffill_nan at 0x7f4e2df7d4c0>),
                                                  ['global_active_power',
                                                   'global_reactive_power',
                                                   'voltage',
                                                   'global_intensity',
                                                   'global_consumption']),
                                                 ('datetime_features_adder',
                                                  FunctionTransformer(func=<function add_datetime_features at 0x7f4e2df7da60>),
                                                  ['date', 'time'])])),
                ('std_scaler', StandardScaler()),
                ('estimator', RandomForestRegressor(n_estimators=10))])

# 3. Prepare the data to train

In [24]:
# Take just a subset of the data for now (year==2008)
data = pd.read_csv('../data/household_power_consumption.txt', sep=';', na_values='?')
data = data[data.Date.str.endswith('2008')]
data.columns = data.columns.str.lower()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 527040 entries, 547596 to 1074635
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   date                   527040 non-null  object 
 1   time                   527040 non-null  object 
 2   global_active_power    526905 non-null  float64
 3   global_reactive_power  526905 non-null  float64
 4   voltage                526905 non-null  float64
 5   global_intensity       526905 non-null  float64
 6   sub_metering_1         526905 non-null  float64
 7   sub_metering_2         526905 non-null  float64
 8   sub_metering_3         526905 non-null  float64
dtypes: float64(7), object(2)
memory usage: 40.2+ MB


In [25]:
labels = ['sub_metering_1', 'sub_metering_2', 'sub_metering_3']
data['global_consumption'] = data[labels].sum(axis=1)

In [26]:
# Create X, y and save datetime in a separeted column
X = data.drop(columns=labels)
y = data[labels]
X.shape, y.shape

((527040, 7), (527040, 3))

In [27]:
y.isna().sum()

sub_metering_1    135
sub_metering_2    135
sub_metering_3    135
dtype: int64

In [28]:
# Remove na values from y
y = y.fillna(method='ffill', axis=0)

In [29]:
X.head()

,date,time,global_active_power,global_reactive_power,voltage,global_intensity,global_consumption
547596,1/1/2008,00:00:00,1.620,0.070,241.25,6.6,18.0
547597,1/1/2008,00:01:00,1.626,0.072,241.74,6.6,18.0
547598,1/1/2008,00:02:00,1.622,0.072,241.52,6.6,18.0
547599,1/1/2008,00:03:00,1.612,0.070,240.82,6.6,18.0
547600,1/1/2008,00:04:00,1.612,0.070,240.80,6.6,18.0


In [30]:
y.head()

,sub_metering_1,sub_metering_2,sub_metering_3
547596,0.0,0.0,18.0
547597,0.0,0.0,18.0
547598,0.0,0.0,18.0
547599,0.0,0.0,18.0
547600,0.0,0.0,18.0


In [31]:
# Train/test Split /!\ For later, if we use sequential models (ARIMA, RNN): see if we consider a TS special train/test split (to keep sequence's logic)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((368928, 7), (158112, 7), (368928, 3), (158112, 3))

In [32]:
X_train.head()

,date,time,global_active_power,global_reactive_power,voltage,global_intensity,global_consumption
799479,23/6/2008,22:03:00,2.238,0.068,238.05,9.4,20.0
798682,23/6/2008,08:46:00,2.108,0.402,236.63,9.0,31.0
861409,5/8/2008,22:13:00,0.080,0.000,239.75,0.2,1.0
668140,24/3/2008,17:04:00,0.498,0.078,238.74,2.4,0.0
874365,14/8/2008,22:09:00,0.080,0.000,241.54,0.2,1.0


# 4. Train and save the pipeline

In [33]:
# Function to save the pipeline locally
def save_pipeline(pipeline: Pipeline = None,
                  params: dict = None,
                  metrics: dict = None) -> None:
    """
    persist trained pipeline, params and metrics
    """

    timestamp = time.strftime("%Y%m%d-%H%M%S")

    print(Fore.BLUE + "\nSave pipeline to local disk..." + Style.RESET_ALL)

    # save params
    if params is not None:
        params_path = os.path.join(LOCAL_REGISTRY_PATH, "params", timestamp + ".pickle")
        with open(params_path, "wb") as file:
            pickle.dump(params, file)

    # save metrics
    if metrics is not None:
        metrics_path = os.path.join(LOCAL_REGISTRY_PATH, "metrics", timestamp + ".pickle")
        with open(metrics_path, "wb") as file:
            pickle.dump(metrics, file)

    # save pipeline
    if pipeline is not None:
        pipeline_path = os.path.join(LOCAL_REGISTRY_PATH, "models", timestamp + ".joblib")
        dump(pipeline, pipeline_path)
         
    print("\n✅ data saved locally")

    return None

In [ ]:
%%time

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Mesure its performance
r2_score = pipeline.score(X_test, y_test)
#r2_score = pipeline.score(X_test[['Voltage']].dropna(), y_test.dropna())

# Save it locally
params = dict(
    # Model parameters
    n_estimators=10,

    # Package behavior
    context="train",

    # Data used to fit
    dataset_start=data.date.iloc[0],
    dataset_end=data.date.iloc[-1]
)

metrics = dict(r2_score=r2_score)
print(metrics)

save_pipeline(pipeline, params, metrics)

{'r2_score': 0.8936800089724227}

Save pipeline to local disk...

✅ data saved locally
CPU times: user 30.7 s, sys: 116 ms, total: 30.8 s
Wall time: 30.8 s


# 5. Load the pipeline and try to predict

In [36]:
# To load a specific local pipeline
# pipeline_path = os.path.join(LOCAL_REGISTRY_PATH, "models", "20221123-170451.joblib")
# pipeline_loaded = load(pipeline_path)
# pipeline_loaded


Load pipeline from local disk...
training_outputs/models
h  /root/code/charlgd/electricity-decipherer/notebooks
r  []


In [ ]:
X_test

,date,time,global_active_power,global_reactive_power,voltage,global_intensity,global_consumption
565992,13/1/2008,18:36:00,5.352,0.052,233.86,23.2,48.0
653389,14/3/2008,11:13:00,2.404,0.322,238.22,10.4,17.0
796994,22/6/2008,04:38:00,0.136,0.000,239.50,0.6,0.0
797512,22/6/2008,13:16:00,0.438,0.366,239.43,2.4,1.0
920223,15/9/2008,18:27:00,1.418,0.000,238.58,6.0,18.0
...,...,...,...,...,...,...,...
931737,23/9/2008,18:21:00,0.524,0.000,238.56,2.2,1.0
714716,26/4/2008,01:20:00,0.324,0.098,241.76,1.4,2.0
609776,13/2/2008,04:20:00,0.314,0.082,246.34,1.2,0.0
870016,11/8/2008,21:40:00,0.174,0.148,239.17,0.8,0.0


In [ ]:
y_pred = pd.DataFrame(pipeline_loaded.predict(X_test), columns=labels)
y_pred['datetime'] = pd.to_datetime(X_test['date'] + ' ' + X_test['time'], format="%d/%m/%Y %H:%M:%S")
y_pred

AttributeError: 'NoneType' object has no attribute 'predict'

In [ ]:
pwd

'/root/code/charlgd/electricity-decipherer/notebooks'